In [52]:
#Install dependencies 
!pip install fastapi
!pip install uvicorn
!pip install gradio
!pip install datasets 
!pip install evaluate

!pip install accelerate -U
!pip install pytorch-accelerated
!pip install accelerate[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 KB 412.9 kB/s eta 0:00:000:00:010:00:01:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-accelerated 0.1.49 requires accelerate==0.29.3, but you have accelerate 0.32.1 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#if you are running this in google colab then you dont need to run this. These libraries come preinstalled with google colab
!pip install -r requirements.txt

In [2]:
#necessary imports for training
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk # install this using pip install datasets, this does not come pre installed with colab
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import accelerate

import warnings
warnings.filterwarnings("ignore")

/home/raccoon/Desktop/development/plants/Plant-Disease-Detection/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#checking if cudda is available, it might take a long time on the CPU, please change your runtime to GPU on google colab if the output of this cell is false
torch.cuda.is_available()

True

In [4]:
#if you dont have a GPU, then this code with run on CPU.
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
#setting up some variables
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to(device) #device agnostic

In [7]:
# Preparing the dataset for training
dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')
dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')
dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')

dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text'])
dataset_tts_train = dataset_tts_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text'])
dataset_tts_test  = dataset_tts_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')
dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
dataset_tts_val   = dataset_tts_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

dataset = DatasetDict({ 'train': interleave_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train]),
                        'test': interleave_datasets([dataset_scc_test, dataset_tts_test, dataset_ks_test]),
                        'validation': interleave_datasets([dataset_scc_val, dataset_tts_val, dataset_ks_val])})

In [8]:
# Viewing a single example from the dataset
dataset['test'][99]


{'answer': 'SELECT competition FROM table_name_95 WHERE date = "october 7, 2011"',
 'question': 'What competition was played on October 7, 2011?',
 'context': 'CREATE TABLE table_name_95 (competition VARCHAR, date VARCHAR)'}

In [10]:
# Maping the dataset and defining our tokenize function
def tokenize_function(example):

#     print(len(example["question"]))
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(example['context'], example['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
#     print(prompt[0])
#     print()

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("Loaded Tokenized Dataset")
except:
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer'])

    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("Tokenized and Saved Dataset")

Loaded Tokenized Dataset


In [11]:
# checking out tokenized version of the dataset
print(tokenized_datasets.keys())
print(tokenized_datasets['train'][0].keys())
print(tokenized_datasets['train'][0]['input_ids'][:10])
print(tokenized_datasets['train'][0]['labels'][:10])
print(tokenized_datasets)

dict_keys(['train', 'test', 'validation'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]
[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


In [12]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (118695, 2)
Validation: (14838, 2)
Test: (14835, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


In [13]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
    finetuned_model = finetuned_model.to(device)
    to_train = False

except:
    to_train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
    finetuned_model = finetuned_model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#training cell, this will take a while depending on your hardware, it took me several minutes on a RTX 4090
%%time

if to_train:
    output_dir = f'./sql-training-{str(int(time.time()))}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,                     # number of steps between evaluation
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
    )

    trainer.train()

    finetuned_model.save_pretrained("finetuned_model_2_epoch")

Step,Training Loss,Validation Loss


In [5]:
#loading the trained model, if for some reason model is not trained, lets say because of hardware limitations, then we can load the model from the saved version
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
finetuned_model = finetuned_model.to(device)

In [16]:
#inferencing on the test set
index = 0
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)

Question:
On what Date did the Away team essendon play?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT date FROM table_name_11 WHERE away_team = "essendon"

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
SELECT date FROM table_name_11 WHERE away_team = "essendon"


In [17]:
# Perform inferences for test dataset. Do 25 only, due to time it takes.

questions = dataset['test'][0:25]['question']
contexts = dataset['test'][0:25]['context']
human_baseline_answers = dataset['test'][0:25]['answer']

original_model_answers = []
finetuned_model_answers = []

for idx, question in enumerate(questions):

    prompt = f"""Tables:
{contexts[idx]}

Question:
{question}

Answer:
"""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)

    human_baseline_text_output = human_baseline_answers[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_answers.append(original_model_text_output)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_text_output)

zipped_summaries = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_answers', 'original_model_answers', 'finetuned_model_answers'])
# df

Token indices sequence length is longer than the specified maximum sequence length for this model (1115 > 512). Running this sequence through the model will result in indexing errors


In [18]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_answers,
    references=human_baseline_answers[0:len(original_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)


finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=human_baseline_answers[0:len(finetuned_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results)

ORIGINAL MODEL:
{'rouge1': 0.03194876200890416, 'rouge2': 0.0054571428571428575, 'rougeL': 0.03174603174603174, 'rougeLsum': 0.03108333984222448}
FINE-TUNED MODEL:
{'rouge1': 0.6572020416366311, 'rouge2': 0.5613417581194815, 'rougeL': 0.647988094471915, 'rougeLsum': 0.6471310368010181}


In [53]:
# Converting the code to a function to be used repeatedly
import json

def generate_sql_from_text(question, context, tokenizer, finetuned_model, device):
    """
    Generate an SQL query from natural language text using a fine-tuned model.
    
    Parameters:
        question (str): The question in natural language.
        context (str): The context or tables information in natural language.
        tokenizer (object): The tokenizer to process the input.
        finetuned_model (object): The fine-tuned model to generate the SQL query.
        device (str): The device to use for computation ('cpu' or 'cuda').
        
    Returns:
        dict: A dictionary containing the input prompt and the model-generated SQL query.
    """
    prompt = f"""Tables:
    {context}

    Question:
    {question}

    SQL Query:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = inputs.to(device)

    output = tokenizer.decode(
        finetuned_model.generate(
            inputs["input_ids"],
            max_new_tokens=200,
        )[0],
        skip_special_tokens=True
    )

    result = {
        "input_prompt": prompt,
        "generated_sql_query": output
    }

    return result


In [48]:
# function inference for the fine-tuned model
question = "What is the average salary of employees in the Sales department?"
context = "Table: Employees (EmployeeID, Name, Department, Salary)"
result= generate_sql_from_text(question, context, tokenizer, finetuned_model, device)
print(result)


{'input_prompt': 'Tables:\n    Table: Employees (EmployeeID, Name, Department, Salary)\n\n    Question:\n    What is the average salary of employees in the Sales department?\n\n    SQL Query:\n    ', 'generated_sql_query': 'SELECT DISTINCT T1.Name, T1.Name FROM Employee AS T1 JOIN Employee AS T2 ON T1.EmployeeID = T2.EmployeeID GROUP BY T1.Name ORDER BY T1.Name'}


In [54]:
# for running it the fast api, please run api.py module with its args, 
# port= 0000 choose any port number to run it on
# method= [api,gradio] 